In [2]:
import requests

In [6]:
r = requests.get(url="https://www.cbioportal.org/api/health")
print (r.json())
print (r.status_code)

{'status': 'UP'}
200


Part 1: Retrieve and display patient data. User must provide a valid patient and study Id. 
Example uses
patient: P-0000004 
study: msk_impact_2017

In [7]:
r = requests.get(url="https://www.cbioportal.org/api/studies/msk_impact_2017/patients/P-0000004/clinical-data")
print (r.json())
# look for nicer way to print than just a straight line of key/val pairs

[{'uniquePatientKey': 'UC0wMDAwMDA0Om1za19pbXBhY3RfMjAxNw', 'patientId': 'P-0000004', 'studyId': 'msk_impact_2017', 'clinicalAttributeId': 'OS_STATUS', 'value': '0:LIVING'}, {'uniquePatientKey': 'UC0wMDAwMDA0Om1za19pbXBhY3RfMjAxNw', 'patientId': 'P-0000004', 'studyId': 'msk_impact_2017', 'clinicalAttributeId': 'SAMPLE_COUNT', 'value': '1'}, {'uniquePatientKey': 'UC0wMDAwMDA0Om1za19pbXBhY3RfMjAxNw', 'patientId': 'P-0000004', 'studyId': 'msk_impact_2017', 'clinicalAttributeId': 'SEX', 'value': 'Female'}, {'uniquePatientKey': 'UC0wMDAwMDA0Om1za19pbXBhY3RfMjAxNw', 'patientId': 'P-0000004', 'studyId': 'msk_impact_2017', 'clinicalAttributeId': 'SMOKING_HISTORY', 'value': 'Unknown'}, {'uniquePatientKey': 'UC0wMDAwMDA0Om1za19pbXBhY3RfMjAxNw', 'patientId': 'P-0000004', 'studyId': 'msk_impact_2017', 'clinicalAttributeId': 'VITAL_STATUS', 'value': 'ALIVE'}]
